# Harvey CDBG-DR Funding Analysis

This notebook analyzes Hurricane Harvey disaster recovery funding flows from HUD allocations to local expenditures.

**Data Sources:**
- 106 Harvey DRGR reports (2018-2025)
- National grants data for comparison

**Key Questions:**
1. Where did Harvey funds end up vs. allocations in Action Plans?
2. How have funding allocations changed over time?
3. What is the completion rate by program type and geography?

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import sqlite3
import json
from pathlib import Path

from harvey_queries import HarveyQueries
from funding_tracker import FundingTracker

# Initialize query interface
hq = HarveyQueries()

print("Harvey Query Interface loaded successfully")

## 1. Funding Summary

In [ ]:
# Get overall summary
summary = hq.get_summary()

print(f"Quarter: {summary['quarter']}")
print(f"\nTotal Budget: ${summary['total_budget']:,.2f}")
print(f"Activities: {summary['activity_count']}")
print(f"  - Completed: {summary['completed']}")
print(f"  - In Progress: {summary['in_progress']}")
print(f"\nOrganizations: {summary['organizations']}")
print(f"Counties: {summary['counties']}")

print("\nBy Program:")
for prog, data in summary['by_program'].items():
    print(f"  {prog}: ${data['budget']:,.2f} ({data['count']} activities)")

## 2. Allocation vs Expenditure by Program

In [ ]:
# Get allocation vs expenditure comparison
df_alloc = hq.get_allocation_vs_expenditure()
df_alloc

In [ ]:
# Visualize program allocation
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Budget by program
ax1 = axes[0]
df_alloc.plot(kind='bar', x='program_type', y='allocated', ax=ax1, legend=False)
ax1.set_title('Total Budget by Program Type')
ax1.set_xlabel('Program Type')
ax1.set_ylabel('Budget ($)')
ax1.ticklabel_format(style='plain', axis='y')

# Activity counts by status
ax2 = axes[1]
df_alloc.plot(kind='bar', x='program_type', 
              y=['completed_count', 'in_progress_count', 'cancelled_count'], 
              ax=ax2, stacked=True)
ax2.set_title('Activity Status by Program Type')
ax2.set_xlabel('Program Type')
ax2.set_ylabel('Activity Count')
ax2.legend(['Completed', 'In Progress', 'Cancelled'])

plt.tight_layout()
plt.show()

## 3. Funding by Organization

In [ ]:
# Get organization-level funding
df_orgs = hq.get_funding_by_organization()
df_orgs.head(10)

In [ ]:
# Top 10 organizations by funding
top_orgs = df_orgs.groupby('organization')['allocated'].sum().sort_values(ascending=False).head(10)

fig, ax = plt.subplots(figsize=(10, 6))
top_orgs.plot(kind='barh', ax=ax)
ax.set_title('Top 10 Organizations by Funding Allocation')
ax.set_xlabel('Allocated ($)')
ax.set_ylabel('Organization')
plt.tight_layout()
plt.show()

## 4. Geographic Distribution

In [ ]:
# Get county-level funding
df_counties = hq.get_funding_by_county()
df_counties = df_counties[df_counties['county'] != 'Statewide']
df_counties.head(20)

In [ ]:
# Top 15 counties by funding
top_counties = df_counties.groupby('county')['allocated'].sum().sort_values(ascending=False).head(15)

fig, ax = plt.subplots(figsize=(10, 8))
top_counties.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Top 15 Counties by Funding Allocation')
ax.set_xlabel('Allocated ($)')
ax.set_ylabel('County')
plt.tight_layout()
plt.show()

## 5. Quarterly Trends

In [ ]:
# Get quarterly trends
df_trends = hq.get_quarterly_trends()
df_trends.head()

In [ ]:
# Plot quarterly budget trends
pivot = df_trends.pivot_table(index='quarter', columns='program_type', 
                               values='total_budget', aggfunc='sum')

fig, ax = plt.subplots(figsize=(14, 6))
pivot.plot(ax=ax, marker='o')
ax.set_title('Quarterly Budget Trends by Program Type')
ax.set_xlabel('Quarter')
ax.set_ylabel('Total Budget ($)')
ax.legend(title='Program Type')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Completion Rates

In [ ]:
# Get completion rates by category
df_completion = hq.get_completion_rates()
df_completion

In [ ]:
# Completion rates visualization
fig, ax = plt.subplots(figsize=(10, 6))

df_completion_sorted = df_completion.sort_values('completion_rate', ascending=True)
colors = ['green' if x > 50 else 'orange' if x > 20 else 'red' 
          for x in df_completion_sorted['completion_rate']]

ax.barh(df_completion_sorted['activity_category'], 
        df_completion_sorted['completion_rate'],
        color=colors)
ax.set_title('Completion Rate by Activity Category')
ax.set_xlabel('Completion Rate (%)')
ax.set_ylabel('Activity Category')
ax.axvline(x=50, color='gray', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## 7. Sankey Diagram Data

In [ ]:
# Load Sankey data
with open('../outputs/exports/harvey/harvey_sankey_data.json') as f:
    sankey = json.load(f)

print(f"Quarter: {sankey['quarter']}")
print(f"\nSummary:")
print(f"  Total Budget: ${sankey['summary']['total_budget']:,.2f}")
print(f"  Nodes: {len(sankey['nodes'])}")
print(f"  Links: {len(sankey['links'])}")

print("\nNodes:")
for node in sankey['nodes']:
    print(f"  Level {node['level']}: {node['name']}")

print("\nTop 10 Flows:")
for link in sorted(sankey['links'], key=lambda x: x['value'], reverse=True)[:10]:
    print(f"  {link['source']} -> {link['target']}: ${link['value']:,.2f}")

## 8. National Grants Comparison

Compare locally tracked Harvey activities to national CDBG-DR grant data.

In [ ]:
# Connect to database for national comparison
conn = sqlite3.connect('../data/glo_reports.db')

# Get national grants for Harvey
national = pd.read_sql_query("""
    SELECT disaster_type, program_type, 
           total_obligated, total_expended,
           ratio_expended_obligated
    FROM national_grants
    WHERE disaster_type LIKE '%Harvey%'
""", conn)

print("National Grants Data for Harvey:")
display(national)

# Summary
print(f"\nTotal Obligated: ${national['total_obligated'].sum():,.2f}")
print(f"Total Expended: ${national['total_expended'].sum():,.2f}")
print(f"Overall Expenditure Rate: {national['total_expended'].sum() / national['total_obligated'].sum() * 100:.1f}%")

In [ ]:
# Compare to local tracking
local_summary = hq.get_summary()

print("\nComparison:")
print(f"{'Metric':<30} {'National':>20} {'Local Tracking':>20}")
print("-" * 70)
print(f"{'Total Budget/Obligated':<30} ${national['total_obligated'].sum():>18,.0f} ${local_summary['total_budget']:>18,.0f}")
print(f"{'Coverage':<30} {'':>20} {local_summary['total_budget']/national['total_obligated'].sum()*100:>19.1f}%")

## Summary

This analysis provides:
1. **Funding tracking**: $4.47B in Harvey activities tracked across 616 individual activities
2. **Program breakdown**: Infrastructure dominates at $4.4B vs Housing at $57.8M
3. **Geographic distribution**: Funding distributed across 62 Texas counties
4. **Completion status**: Overall 12.5% completion rate for Infrastructure
5. **Sankey data**: Ready for visualization showing flow from HUD → GLO → Programs → Organizations → Counties

In [ ]:
# Cleanup
hq.close()
conn.close()
print("Analysis complete!")